In [2]:
import numpy as np
from scipy import stats as st
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
# d)
n = 100
theta = 3
sample = np.pow(1 - np.random.rand(n), 1 / (1 - theta))
sample

array([1.41346661, 1.2661022 , 1.21138476, 1.84711006, 2.30617454,
       3.60905164, 1.08802688, 1.25672836, 1.1837064 , 3.94856169,
       1.21371951, 2.16082598, 3.43425602, 5.9864374 , 1.98077684,
       1.18473628, 1.69426348, 1.53096736, 3.40205581, 1.11719655,
       1.93425033, 1.03938543, 1.42123253, 1.50680292, 1.57438131,
       3.80445378, 1.04327992, 1.84269538, 1.38442297, 1.77045252,
       1.65130723, 1.05761993, 1.32280571, 1.23907394, 2.1137118 ,
       1.38901558, 3.79710463, 1.5585524 , 1.02327373, 1.24022571,
       1.03239175, 1.46370859, 1.05399354, 1.09598173, 1.35796643,
       2.97334518, 1.31650443, 1.88764514, 1.05706802, 1.76243688,
       1.42793582, 1.70125254, 1.16166761, 1.70250402, 1.8586598 ,
       1.32402892, 1.74887435, 2.52792164, 1.23337341, 1.8645685 ,
       2.71219858, 5.92469347, 1.05377394, 1.03670778, 2.02841937,
       1.01871727, 1.32977836, 1.20177314, 1.14088525, 3.24237356,
       1.8204553 , 1.4007678 , 1.97318356, 1.4381452 , 1.02234

In [8]:
theta_assessment = (n / np.sum(np.log(sample))) + 1
theta_assessment

np.float64(3.119351125177807)

In [9]:
median_assessment = np.pow(2, 1 / (theta_assessment - 1))
median_assessment

np.float64(1.386879584543782)

In [12]:
theta_asymptotic_ci = (
    float(theta_assessment - 1.96 * (theta_assessment - 1) / n**0.5),
    float(theta_assessment + 1.96 * (theta_assessment - 1) / n**0.5)
)

median_asymptotic_ci = (
    float(-1.96 * median_assessment * np.log(2) / (theta_assessment - 1) / np.sqrt(n) + median_assessment),
    float(1.96 * median_assessment * np.log(2) / (theta_assessment - 1) / np.sqrt(n) + median_assessment)
)

print(
    f"{theta_asymptotic_ci = }",
    f"{median_asymptotic_ci = }",
    sep="\n\n"
)

theta_asymptotic_ci = (2.703958304642957, 3.5347439457126573)

median_asymptotic_ci = (1.2979763887368503, 1.4757827803507138)


In [ ]:
# t)
from collections.abc import Callable
from typing import NamedTuple


class ConfidenceInterval(NamedTuple):
    low: float
    high: float


def parametric_bootstrap(
    data: np.ndarray,
    statistic: Callable[[np.ndarray], float],
    prob_model: Callable[[int], np.ndarray],
    n_resamples: int = 10_000,
    confidence_level: float = 0.95,
) -> ConfidenceInterval:
    n = len(data)
    bootstrap_data = np.ndarray((n_resamples, ), dtype=np.float32)

    for i in range(n_resamples):
        resample = prob_model(n)
        bootstrap_data[i] = statistic(resample)
    
    bootstrap_data.sort()

    return ConfidenceInterval(
        low=bootstrap_data[round((1 - confidence_level) / 2 * n_resamples) - 1],
        high=bootstrap_data[round((1 + confidence_level) / 2 * n_resamples) - 1]
    )


statistic = lambda x: (n / np.sum(np.log(x))) + 1 - theta_assessment

theta_nonparametric_bootstrap_ci = st.bootstrap(
    (sample, ), 
    statistic, 
    n_resamples=1_000, 
    method='basic',
    confidence_level=0.95
).confidence_interval
theta_nonparametric_bootstrap_ci = (
    float(theta_assessment - theta_nonparametric_bootstrap_ci.high),
    float(theta_assessment - theta_nonparametric_bootstrap_ci.low)
)

theta_parametric_bootstrap_ci = parametric_bootstrap(
    sample,
    statistic,
    lambda n: np.pow(np.random.rand(n), 1 / (1 - theta_assessment)),
    n_resamples=50_000, 
    confidence_level=0.95
)
theta_parametric_bootstrap_ci = (
    float(theta_assessment - theta_parametric_bootstrap_ci.high),
    float(theta_assessment - theta_parametric_bootstrap_ci.low)
)

print(
    f"{theta_nonparametric_bootstrap_ci = }",
    f"{theta_parametric_bootstrap_ci = }",
    sep="\n\n"
)

theta_nonparametric_bootstrap_ci = (2.8188381132143965, 3.5544267052831864)

theta_parametric_bootstrap_ci = (2.632987863239712, 3.4770670596928066)


In [19]:
statistic = lambda x: np.pow(2, 1 / (n / np.sum(np.log(x)))) - median_assessment

median_nonparametric_bootstrap_ci = st.bootstrap(
    (sample, ), 
    statistic, 
    n_resamples=1_000,
    method='basic',
    confidence_level=0.95
).confidence_interval
median_nonparametric_bootstrap_ci = (
    float(median_assessment - median_nonparametric_bootstrap_ci.high),
    float(median_assessment - median_nonparametric_bootstrap_ci.low)
)

median_parametric_bootstrap_ci = parametric_bootstrap(
    sample,
    statistic,
    lambda n: np.pow((1 - np.random.rand(n)), 1 / (1 - theta_assessment)),
    n_resamples=50_000, 
    confidence_level=0.95
)
median_parametric_bootstrap_ci = (
    float(median_assessment - median_parametric_bootstrap_ci.high),
    float(median_assessment - median_parametric_bootstrap_ci.low)
)

print(
    f"{median_nonparametric_bootstrap_ci = }",
    f"{median_parametric_bootstrap_ci = }",
    sep="\n\n"
)

median_nonparametric_bootstrap_ci = (1.3212664361761504, 1.4697341535265536)

median_parametric_bootstrap_ci = (1.2899195614982422, 1.4690410781431968)


In [ ]:
# f)
print(
    f"{theta_asymptotic_ci = }\nl = {theta_asymptotic_ci[1] - theta_asymptotic_ci[0]}",
    f"{theta_nonparametric_bootstrap_ci = }\nl = {theta_nonparametric_bootstrap_ci[1] - theta_nonparametric_bootstrap_ci[0]}",
    f"{theta_parametric_bootstrap_ci = }\nl = {theta_parametric_bootstrap_ci[1] - theta_parametric_bootstrap_ci[0]}",
    sep="\n\n"
)

theta_asymptotic_ci = (2.703958304642957, 3.5347439457126573)
l = 0.8307856410697001

theta_nonparametric_bootstrap_ci = (2.8188381132143965, 3.5544267052831864)
l = 0.7355885920687899

theta_parametric_bootstrap_ci = (2.632987863239712, 3.4770670596928066)
l = 0.8440791964530945


In [21]:
print(
    f"{median_asymptotic_ci = }\nl = {median_asymptotic_ci[1] - median_asymptotic_ci[0]}",
    f"{median_nonparametric_bootstrap_ci = }\nl = {median_nonparametric_bootstrap_ci[1] - median_nonparametric_bootstrap_ci[0]}",
    f"{median_parametric_bootstrap_ci = }\nl = {median_parametric_bootstrap_ci[1] - median_parametric_bootstrap_ci[0]}",
    sep="\n\n"
)

median_asymptotic_ci = (1.2979763887368503, 1.4757827803507138)
l = 0.17780639161386347

median_nonparametric_bootstrap_ci = (1.3212664361761504, 1.4697341535265536)
l = 0.1484677173504032

median_parametric_bootstrap_ci = (1.2899195614982422, 1.4690410781431968)
l = 0.17912151664495468
